In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import bs4
import re

## For a player

In [6]:
driver = webdriver.Edge()
driver.get('https://qlstats.net/player/284886')

### Cookie screen

In [7]:
def is_cookie_screen(driver : selenium.webdriver):
    '''A function to check if the given webpage is the 'accept cookies' screen.
    Regex matches the body of the '''
    element = driver.find_element(By.TAG_NAME, value = 'body')
    cookie_screen = re.compile(r'.*(To continue using qlstats, you need to agree to the use of cookies.\nAgree).*')
    if cookie_screen.search(element.text) is None:
        return False
    return True


In [8]:
#press the button
if is_cookie_screen(driver):
    try:
        button = driver.find_element(By.TAG_NAME, 'button')
        button.click()
    except:
        "Cookies could not be accepted, please recheck"

### Scraping the player page